In [292]:
import json
import os
import numpy
import pandas
import pyarrow
import sys

from datetime import date
from os import path
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [293]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [294]:
COMPANY_NAME = 'BBF'
COMPANY_IDENTIFIER = 'BBF'
TRANSFER_PACKAGES_START_DATE = '2019-01-01'
SALES_TRANSACTIONS_START_DATE = '2019-01-01'
INVENTORY_DATES = [
    '09/30/2020',
    '10/31/2020',
    '11/30/2020',
    '12/31/2020',
    '01/31/2021',
    '02/28/2021',
    '03/31/2021',
    '04/30/2021',
    '05/31/2021',
    '06/30/2021',
    '07/31/2021',
    '08/31/2021',
    '09/30/2021',
]
ANALYSIS_PARAMS = {
    'sold_threshold': 1.0
}
TODAY_DATE = date.today().strftime('%m/%d/%Y')
INVENTORY_DATES.append(TODAY_DATE)
print('Today is {}'.format(TODAY_DATE))

Today is 10/31/2021


In [295]:
# Download packages, sales transactions, incoming / outgoing tranfers

company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(
    COMPANY_IDENTIFIER, include_quantity_zero=True)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [296]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from util import active_inventory_util as util

In [323]:
d = util.Download()
d.download_dataframes(
    incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
    outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
    sales_transactions_dataframe=company_sales_transactions_dataframe,
    engine=engine
)

WARN: package 1404181 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 882535 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 943375 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 939603 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1480915 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1143130 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 999491 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1041172 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1063602 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 999497 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1309319 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: package 1089736 is missing a sourceproductionbatchnumber and an incoming pkg
WARN: pac

In [325]:
len(d.parent_packages_records)

11

In [320]:
production_batch_numbers = set([])
for pkg in d.missing_incoming_pkg_package_records:
    production_batch_numbers.add(pkg['package_payload']['sourceproductionbatchnumbers'])

In [335]:
# Child to parent package_id

productionbatchnum_to_package_id = {}
child_to_parent_package_id = {}

for parent_record in d.parent_packages_records:
    production_batch_num = parent_record['package_payload']['productionbatchnumber']
    package_id = parent_record['package_id']
    if not production_batch_num:
        print(f'Parent package {package_id} is missing a productionbatchnumber')
        continue
    productionbatchnum_to_package_id[parent_record['package_payload']['productionbatchnumber']] = package_id

num_orphans = 0
has_number_but_no_parent = 0

for pkg in d.missing_incoming_pkg_package_records:
    batch_no = pkg['package_payload']['sourceproductionbatchnumbers']
    if not batch_no:
        num_orphans += 1
        continue
    
    if batch_no not in productionbatchnum_to_package_id:
        num_orphans += 1
        has_number_but_no_parent += 1
        print(batch_no)
        continue
    
    parent_package_id = productionbatchnum_to_package_id[batch_no]
    child_to_parent_package_id[pkg['package_id']] = parent_package_id
    

SFVM22103312-1
BRYM22103312-1
TOGM22101272-4
WDCM12103282-1
TOGM22101272-4
BRYM22103312-1
BRYM22103312-1
WDCM12103282-1
WDCM12103282-1
SFVM22103312-1
BRYM22103312-1
MACM22103312-6
MACM22103312-6


In [330]:
len(child_to_parent_package_id.keys())

31

In [332]:
num_orphans

174

In [334]:
has_number_but_no_parent

13

In [310]:
# Check for how many packages are excluded from analysis,
# seeing less then 2% is really good
q = util.Query()
q.inventory_dates = INVENTORY_DATES
q.company_name = COMPANY_NAME

id_to_history = util.get_histories(d)
util.print_counts(id_to_history)
util.create_inventory_xlsx(id_to_history, q, params=ANALYSIS_PARAMS)

Only outgoing: 43
Only incoming: 136
Only sold: 205
In and out: 25
In and sold at least once 533
In and sold many times 510
Total pkgs: 930
Wrote result to out/BBF_inventory_by_month.xls
Excluded 248 / 930 packages from consideration (26.67%)
  MISSING_INCOMING: 248 times


In [ ]:
# Find parent - child relationship information, e.g.,
# 1. Search for all packages which match based on sourceproductionbatchnumbers
#
# 2. Match these to metrc_packages.productionbatchnumber
#
# 3. Create an incoming pkg for the PackageHistory
#.    where sourceproductionbatchnumber = metrc_packages.productionbatchnumber

In [289]:
import importlib
importlib.reload(util)

INVENTORY_DATES = [TODAY_DATE]

date_to_inventory_packages_dataframe = {}

id_to_history = util.get_histories(d)

for inventory_date in INVENTORY_DATES:
    computed_inventory_package_records = util.create_inventory_dataframe_by_date(
        id_to_history, inventory_date, params=ANALYSIS_PARAMS)    
    computed_inventory_packages_dataframe = pandas.DataFrame(
        computed_inventory_package_records,
        columns=[
            'package_id',
            'license_number',
            'arrived_date',
            'product_category_name',
            'product_name',
            'quantity',
            'sold_date',
            'is_in_inventory'
        ]
    )
    date_to_inventory_packages_dataframe[inventory_date] = computed_inventory_packages_dataframe

WARN: package #5172609 does not have a shipped quantity
WARN: package #848108 does not have a shipped quantity


In [302]:
from_packages_inventory_dataframe = company_inventory_packages_dataframe[[
    'package_id',
    'packaged_date',
    'product_category_name',
    'product_name',
    'quantity',
]].sort_values('package_id')

package_id_to_actual_row = {}
for index, row in from_packages_inventory_dataframe.iterrows():
    package_id_to_actual_row[str(row['package_id'])] = row

In [309]:
# For debugging individual package histories
# You have to run the above block to reload the package_id_to_history array

import importlib
importlib.reload(util)

# Missing transactions?
# 5189649

# Using grams or missing transactions?
#WARN: seeing an incoming package for #19083030 with no received_datetime
#WARN: seeing an incoming package for #19087962 with no received_datetime
#WARN: seeing an incoming package for #19083120 with no received_datetime
#WARN: seeing an incoming package for #19083028 with no received_datetime
#Exception: FATAL error, could not find a transfer to insert a tx with date
#2021-10-22 11:58:45+00:00 into for package 18218608

PACKAGE_IDS = [
    '622839'
]

util.analyze_specific_package_histories(
    d, package_id_to_actual_row, PACKAGE_IDS, params=ANALYSIS_PARAMS)

! Missing in metrc_packages
INCOMING
{'package_row_id': 'b31568a5-0c54-4ecc-89bf-bb62b1b11cea', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'MR281525', 'manifest_number': '0000198704', 'created_date': datetime.date(2020, 8, 4), 'received_datetime': datetime.datetime(2020, 8, 4, 17, 23, 9, tzinfo=<UTC>), 'shipper_facility_license_number': 'MP281303', 'shipper_facility_name': 'SIRA NATURALS, INC.', 'recipient_facility_license_number': 'MR281525', 'recipient_facility_name': 'Boston Bud Factory Inc.', 'shipment_type_name': 'Unaffiliated Transfer', 'shipment_transaction_type': 'Wholesale', 'package_id': '622839', 'package_label': '1A40A030000025A000013437', 'type': 'transfer', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Infused (edible)', 'product_name': 'M00000247627: Wicked Sour Watermelon Gummies 5mg 20pk', 'package_lab_results_status': 'passed', 'shipper_wholesale_price': 0.01, 'shipped_quantity': 6

In [291]:
import importlib
importlib.reload(util)

res = util.compare_inventory_dataframes(
    computed=date_to_inventory_packages_dataframe[TODAY_DATE],
    actual=from_packages_inventory_dataframe,
    options={
        'num_errors_to_show': 10,
        'accept_computed_when_sold_out': True
    }
)

# Two categories for: Num actual packages not computed: 229
# 1. Actual has a package that we've never seen
# 2. Actual has a package that we've sold out of, and that's why it's in the inventory

# Two categories for: Num computed packages not in actual: 237
# 1. We never saw the package in the actual inventory
# 2. Actual inventory is sold out, but we think it's not sold out in computed

Pct of # inventory matching: 90.92% (2052 / 2257)
Accuracy of quantities: 98.91%
Pct of # inventory packages over-estimated: 2.61%
Pct of # quantity over-estimated: 0.04%
Avg quantity delta: 0.13
Avg quantity: 11.81

Num matching packages: 2052
Num actual packages not computed: 205
  but computed at some point: 0, e.g., 0.00% of non-computed packages
  avg quantity from actual packages 0.00
Num computed packages not in actual: 59
  but in actual inventory at some point: 19

Computed has these extra package IDs; first 10
4465593; computed quantity 185
15863661; computed quantity 71
3987073; computed quantity 65
9804525; computed quantity 64
17539625; computed quantity 52
17539602; computed quantity 52
17539609; computed quantity 52
17539613; computed quantity 52
17539620; computed quantity 52
9804668; computed quantity 50
4426074; computed quantity 46

Computed is missing these package IDs; first 10
18733266; quantity: 95.0
18978158; quantity: 92.0
18733189; quantity: 77.0
18673122; qua

In [270]:
inactive_packages_df = pandas.read_sql_query(
    util.are_packages_inactive_query(['766095']),
    engine
)
inactive_packages_df

,identifier,license_number,type,package_id,package_label,product_category_name,product_name,archiveddate,finisheddate,quantity
0,HPCC,C10-0000064-LIC,inactive,766095,1A406030000339D000000028,Leaf,wedding crasher trim,None,2020-02-22,0.0


In [273]:
# Find transfer packages in data warehouse by package_id.

def create_transfer_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            company_deliveries.delivery_type,
            company_deliveries.updated_at,
            metrc_transfer_packages.package_id,
            metrc_transfer_packages.package_label,
            metrc_transfer_packages.shipped_quantity,
            metrc_transfer_packages.shipper_wholesale_price,
            metrc_transfers.shipper_facility_name,
            metrc_transfers.shipper_facility_license_number,
            metrc_deliveries.recipient_facility_name,
            metrc_deliveries.recipient_facility_license_number
        from
            metrc_transfer_packages
            inner join metrc_deliveries on metrc_transfer_packages.delivery_row_id = metrc_deliveries.id
            inner join metrc_transfers on metrc_deliveries.transfer_row_id = metrc_transfers.id
            inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
            inner join companies on company_deliveries.company_id = companies.id
        where
            True
            and metrc_transfer_packages.package_id = '{package_id}'
    """

transfer_packages_by_package_id_query = create_transfer_packages_by_package_id_query(
    766095)
transfer_packages_by_package_id_dataframe = pandas.read_sql_query(transfer_packages_by_package_id_query, engine)
transfer_packages_by_package_id_dataframe

,identifier,delivery_type,updated_at,package_id,package_label,shipped_quantity,shipper_wholesale_price,shipper_facility_name,shipper_facility_license_number,recipient_facility_name,recipient_facility_license_number
0,HPCC,INCOMING_UNKNOWN,2021-09-19 01:10:07.977000+00:00,766095,1A406030000339D000000028,7500.0,900.0,"GREEN SPECTRUM TRADING, INC.",C11-0000760-LIC,"HUENEME PATIENT CONSUMER COLLECTIVE, LLC.",C10-0000064-LIC


In [18]:
# Find packages in data warehouse by package_id.

def create_packages_by_package_id_query(package_id):
    return f"""
        select
            companies.identifier,
            metrc_packages.package_id,
            metrc_packages.package_label,
            metrc_packages.quantity
        from
            metrc_packages
            inner join companies on metrc_packages.company_id = companies.id
        where
            True
            and metrc_packages.package_id = '{package_id}'
    """

packages_by_package_id_query = create_packages_by_package_id_query(17481729)
packages_by_package_id_dataframe = pandas.read_sql_query(packages_by_package_id_query, engine)
packages_by_package_id_dataframe

,identifier,package_id,package_label,quantity
0,RA,17481729,1A4060300003BC9000058384,1.0



Package 5751573 arrived on 07/13/2020 with quantity 280 and price $350.0. {'package_row_id': '4544aa24-1a3d-494b-b312-de17db773e79', 'delivery_type': 'INCOMING_FROM_VENDOR', 'license_number': 'C10-0000596-LIC', 'manifest_number': '0000767939', 'created_date': datetime.date(2020, 7, 13), 'received_datetime': Timestamp('2020-07-15 00:56:06+0000', tz='UTC'), 'shipper_facility_license_number': 'C11-0000349-LIC', 'shipper_facility_name': 'MISSION HILLS PATIENTS COLLECTIVE', 'recipient_facility_license_number': 'C10-0000596-LIC', 'recipient_facility_name': 'ROYAL APOTHECARY L.L.C.', 'shipment_type_name': 'Wholesale Manifest', 'shipment_transaction_type': 'Wholesale', 'package_id': '5751573', 'package_label': '1A4060300003D59000009129', 'type': 'transfer_incoming', 'shipment_package_state': 'Accepted', 'is_testing_sample': False, 'is_trade_sample': False, 'product_category_name': 'Flower', 'product_name': 'Rich & Ruthless - CPT Kush Ride (28 GM Flower - SMS) (#112-CPTK-YF0420) #1884', 'packa

In [61]:
from bespoke.metrc.common import metrc_common_util

rest = metrc_common_util.get_rest_helper_for_debug(
    us_state='CA',
    license_number='C10-0000596-LIC'
)

In [62]:
resp = rest.get('/sales/v1/receipts/36748697')

In [66]:
json.loads(resp.content)

{'Id': 36748697,
 'ReceiptNumber': '0036748697',
 'SalesDateTime': '2020-07-18T15:48:25.790',
 'SalesCustomerType': 'Consumer',
 'PatientLicenseNumber': '',
 'CaregiverLicenseNumber': '',
 'IdentificationMethod': '',
 'TotalPackages': 1,
 'TotalPrice': 62.55,
 'Transactions': [{'PackageId': 5751573,
   'PackageLabel': '1A4060300003D59000009129',
   'ProductName': 'Rich & Ruthless - CPT Kush Ride (28 GM Flower - SMS) (#112-CPTK-YF0420) #1884',
   'ProductCategoryName': 'Flower',
   'ItemStrainName': 'CPT Kush Ride',
   'ItemUnitCbdPercent': None,
   'ItemUnitCbdContent': None,
   'ItemUnitCbdContentUnitOfMeasureName': None,
   'ItemUnitCbdContentDose': None,
   'ItemUnitCbdContentDoseUnitOfMeasureName': None,
   'ItemUnitThcPercent': None,
   'ItemUnitThcContent': None,
   'ItemUnitThcContentUnitOfMeasureName': None,
   'ItemUnitThcContentDose': None,
   'ItemUnitThcContentDoseUnitOfMeasureName': None,
   'ItemUnitVolume': None,
   'ItemUnitVolumeUnitOfMeasureName': None,
   'ItemUnitWe

In [69]:
resp2 = rest.get('/sales/v1/receipts/36803933')

In [70]:
json.loads(resp2.content)

{'Id': 36803933,
 'ReceiptNumber': '0036803933',
 'SalesDateTime': '2020-07-18T19:24:11.890',
 'SalesCustomerType': 'Consumer',
 'PatientLicenseNumber': '',
 'CaregiverLicenseNumber': '',
 'IdentificationMethod': '',
 'TotalPackages': 1,
 'TotalPrice': 62.55,
 'Transactions': [{'PackageId': 5751573,
   'PackageLabel': '1A4060300003D59000009129',
   'ProductName': 'Rich & Ruthless - CPT Kush Ride (28 GM Flower - SMS) (#112-CPTK-YF0420) #1884',
   'ProductCategoryName': 'Flower',
   'ItemStrainName': 'CPT Kush Ride',
   'ItemUnitCbdPercent': None,
   'ItemUnitCbdContent': None,
   'ItemUnitCbdContentUnitOfMeasureName': None,
   'ItemUnitCbdContentDose': None,
   'ItemUnitCbdContentDoseUnitOfMeasureName': None,
   'ItemUnitThcPercent': None,
   'ItemUnitThcContent': None,
   'ItemUnitThcContentUnitOfMeasureName': None,
   'ItemUnitThcContentDose': None,
   'ItemUnitThcContentDoseUnitOfMeasureName': None,
   'ItemUnitVolume': None,
   'ItemUnitVolumeUnitOfMeasureName': None,
   'ItemUnitWe